In [6]:
import os,sys
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
# os.environ["CUDA_VISIBLE_DEVICES"]="0" 

import numpy as np
import pandas as pd
import shutil
import pickle
import tqdm
from tqdm import tqdm
from random import shuffle
from math import log, floor
from keras.utils import *
from keras.utils.generic_utils import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *
from keras.preprocessing.image import *
from multiprocessing import *
import gensim
from gensim.models.word2vec import *
from sklearn.metrics.pairwise import *
from sklearn.model_selection import *
from sklearn.manifold import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn import preprocessing
import sent2vec
import re
import string
import unicodedata as udata
import matplotlib
import matplotlib.pyplot as plt
from collections import Counter
from operator import itemgetter
from collections import OrderedDict
import random

In [66]:
model = sent2vec.Sent2vecModel()
model.load_model('model/sent2vec_o2o_o2m_0523.bin') # sent2vec model
all_df = pd.read_csv('./data/tree-rep-profiles-partial/process2family_df.csv')
api_li = ['LoadLibrary',
'CreateProcess',
'OpenProcess',
'ExitProcess',
'TerminateProcess',
'WinExec',
'CreateRemoteThread',
'CreateThread',
'CopyFile',
'CreateFile',
'DeleteFile',
'RegSetValue',
'RegCreateKey',
'RegDeleteKey',
'RegDeleteValue',
'RegQueryValue',
'RegEnumValue',
'WinHttpConnect',
'WinHttpOpen',
'WinHttpOpenRequest',
'WinHttpReadData',
'WinHttpSendRequest',
# 'WinHttpWriteData', #少了
'InternetOpen',
'InternetConnect',
'HttpSendRequest',
'GetUrlCacheEntryInfo']
# api_li = [x.lower() for x in api_li] #lowrer case?
len(api_li)

26

In [43]:
encode_dict = {}
for i,v in enumerate(api_li):
    encode_dict[v] = i+1
pickle.dump(file=open('data/tree-rep-profiles-partial/encode_dict.pkl','wb'),obj=encode_dict)
encode_dict


{'LoadLibrary': 1,
 'CreateProcess': 2,
 'OpenProcess': 3,
 'ExitProcess': 4,
 'TerminateProcess': 5,
 'WinExec': 6,
 'CreateRemoteThread': 7,
 'CreateThread': 8,
 'CopyFile': 9,
 'CreateFile': 10,
 'DeleteFile': 11,
 'RegSetValue': 12,
 'RegCreateKey': 13,
 'RegDeleteKey': 14,
 'RegDeleteValue': 15,
 'RegQueryValue': 16,
 'RegEnumValue': 17,
 'WinHttpConnect': 18,
 'WinHttpOpen': 19,
 'WinHttpOpenRequest': 20,
 'WinHttpReadData': 21,
 'WinHttpSendRequest': 22,
 'InternetOpen': 23,
 'InternetConnect': 24,
 'HttpSendRequest': 25,
 'GetUrlCacheEntryInfo': 26}

In [91]:
root_dir = './data/tree-rep-profiles-partial/normal/'
fam_dir = next(os.walk(root_dir))[1]
fam_dir = [x.split('_')[0] for x in fam_dir]
av_df_col = []
for name in av_col:
    if name in fam_dir:
        av_df_col.append(name)
# av_df_col = ['profile'] + av_df_col
av_df_col

['1.fakeav',
 '2.fakealert',
 '3.kazy',
 '4.zbot',
 '5.symmi',
 '6.allaple',
 '7.virut',
 '8.ursu',
 '9.solimba',
 '10.zusy',
 '11.virtob',
 '12.loadmoney',
 '13.graftor',
 '14.vobfus',
 '15.barys',
 '16.soft',
 '17.expiro',
 '18.sytro',
 '19.razy',
 '20.hotbar',
 '21.ramnit',
 '22.domaiq',
 '23.sality',
 '24.kryptik',
 '25.installcore',
 '26.lmn',
 '27.softpulse',
 '28.strictor',
 '29.delf',
 '30.mplug',
 '31.kdz',
 '32.browsefox',
 '33.autoit',
 '34.downloa',
 '35.nimnul',
 '36.morstar',
 '37.parite',
 '38.msil',
 '39.elkern',
 '40.tepfer',
 '41.outbrowse',
 '42.sirefef',
 '43.mira',
 '44.screensaver',
 '45.firseria',
 '46.killav',
 '47.somoto',
 '48.rahack',
 '49.upatre',
 '50.brresmon',
 '51.shodi',
 '52.startpage',
 '54.soltern',
 '55.zygug',
 '57.mikey',
 '58.ibryte',
 '59.megasearch',
 '60.conjar',
 '63.jorik',
 '64.installerex',
 '65.shipup',
 '66.bdmj',
 '67.madangel',
 '68.bredo',
 '70.hoax',
 '71.eggnog',
 '72.vbkrypt',
 '74.installrex',
 '75.fesber',
 '76.virlock',
 '77.lol

In [95]:
# pid = profile.split('/')[-1].split('.')[0]
# profile_df = all_df[all_df.profile == pid]
# list(profile_df[av_df_col].values[0])
profile = 'c6d288841b9719a01bd340da55538ce43aaa74265791a41ee493cabde9d2402e_2960.profile''

'./data/tree-rep-profiles_one2many/small_short/6.allaple_0.8/G1478/c6d288841b9719a01bd340da55538ce43aaa74265791a41ee493cabde9d2402e_2960.profile'

In [97]:
def byterep_ans(byterep_list,max_length):
    '''
    Input: byte sequenxe (list of pickles path) 
    '''
    rep_list=[]
    for byterep in byterep_list:
        rep = pickle.load(open(byterep,'rb'))
        if len(rep) < max_length:
            for _ in range(max_length-len(rep)):
                rep.append(0)
        rep_list.append(rep)
    return rep_list

# 寫不同模組 來處理x2 y1 y2
def Sent2vec_emb(root_dir,max_length,model=model,dim=700,encode_dict=encode_dict,av_df_col=av_df_col):
    '''
    Create sent2vec profile embedding
    Input: root_dir=>fam-tree-profile
    Output: 
    '''
    sent_pad = [0]*dim
    all_profiles = []
    all_api_name = []
    all_fam_ans = []
    all_byterep=[]
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir + fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
            byterep_list = [x.split('.')[0]+'_byterep.pickle' for x in hl_list]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            byterep_list = [os.path.join(in_directory, f) for f in byterep_list]
            rep_list = byterep_ans(byterep_list,max_length)
            for profile in hl_list:
                pid = profile.split('/')[-1].split('.')[0]
                profile_df = all_df[all_df.profile == pid]
                fam_ans = list(profile_df[av_df_col].values[0])

                profile_emb = []
                func_name_emb = []
                with open(profile,encoding='ISO 8859-1') as f:
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c]+','', lines)
                lines = lines.splitlines()
                for line in lines:
                    temp = re.sub(dil," ",line) # lower? 跟先前一致
                    temp = temp.split(" ")
                    temp = list(filter(None, temp))
                    temp = ' '.join(temp)
                    func_name = temp.split(' ')[0]
                    if func_name not in api_li:
                        print('=ERROR:=',profile,'=>',temp)
                    func_name_emb.append(encode_dict[func_name])
                    emb = model.embed_sentence(temp)
                    emb = emb[0]
                    profile_emb.append(emb)
                if len(lines) < max_length:
                    for _ in range(max_length-len(lines)):
                        profile_emb.append(sent_pad)
                all_profiles.append(profile_emb)
                all_api_name.append(func_name_emb)
                all_fam_ans.append(fam_ans)
            all_byterep.extend(rep_list)
    all_api_name = pad_sequences(all_api_name,maxlen=max_length,padding='post',value=0)
    return np.array(all_profiles) , np.array(all_api_name) , np.array(all_fam_ans) , np.array(all_byterep)

In [98]:
dil= r"[,.;\-+^()/@#?!&$:{}\\*%~\'\"]+\ *" #等號、底線被保留，注意要跟先前一致
max_length = 213

root_dir = './data/tree-rep-profiles-partial/normal/'
train_emb, train_emb_api, train_fam_ans, train_rep_ans = Sent2vec_emb(root_dir,max_length)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/DEV/'
valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans = Sent2vec_emb(root_dir,max_length)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)

root_dir = './data/tree-rep-profiles-partial/TEST/'
test_emb, test_emb_api,test_fam_ans,test_rep_ans = Sent2vec_emb(root_dir,max_length)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_fam_ans.shape,test_rep_ans.shape)

for val in test_emb:
    if len(val) != max_length:
        print(len(val))

  2%|▏         | 2/88 [00:00<00:04, 18.41it/s]

train of sent2vec vector: (11925, 213, 700) (11925, 213) (11925, 87) (11925, 213)


  3%|▎         | 3/88 [00:00<00:04, 17.67it/s]

valid of sent2vec vector: (336, 213, 700) (336, 213) (336, 87) (336, 213)


100%|██████████| 88/88 [00:08<00:00, 10.57it/s]


test of sent2vec vector: (272, 213, 700) (272, 213) (272, 87) (272, 213)


In [101]:
pickle.dump(file=open('data/tree-rep-profiles-partial/TRAIN_vec.pkl','wb'),protocol=4,obj=(train_emb, train_emb_api, train_fam_ans, train_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/DEV_vec.pkl','wb'),protocol=4,obj=(valid_emb, valid_emb_api,valid_fam_ans,valid_rep_ans))
pickle.dump(file=open('data/tree-rep-profiles-partial/TEST_vec.pkl','wb'),protocol=4,obj=(test_emb, test_emb_api,test_fam_ans,test_rep_ans))